# In-Processing

<img src="../images/in-processing.png" alt="Drawing" style="width: 600px;"/>

In [2]:
from aif360.metrics.classification_metric import ClassificationMetric
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
import joblib

from utils import make_dataset, display_results

warnings.filterwarnings('ignore')

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')

DROP_COLS = ['two_year_recid','compas_score','decile_score','compas_class']
FEATURE_COLS = data.drop(DROP_COLS, axis=1).columns.tolist()

train, test = train_test_split(data, test_size=0.2, random_state=1234)

# Prejudice Remover

## Method

## Pros and Cons

## Materials

In [3]:
from aif360.algorithms.inprocessing import PrejudiceRemover

ground_truth_train = make_dataset(train[FEATURE_COLS], train['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)
ground_truth_test = make_dataset(test[FEATURE_COLS], test['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)

clf = PrejudiceRemover(eta=0.1,sensitive_attr='race')

clf.fit(ground_truth_train)
prediction_test = clf.predict(ground_truth_test)

# # Problem with AIF360 implementation
roc_auc = roc_auc_score(test['two_year_recid'], (prediction_test.labels==0).astype(int))

clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [4]:
joblib.dump((clf_metric,roc_auc), '../results/1.3-prejudice_remover.pkl')
display_results('../results/1.3-prejudice_remover.pkl')

,metric_names,scores
0,roc_auc_score,0.651201
1,true_positive_rate_difference,0.100579
2,false_positive_rate_difference,0.230618
3,false_omission_rate_difference,0.035059
4,false_discovery_rate_difference,0.029225
5,error_rate_difference,0.028258
6,false_positive_rate_ratio,1.654547
7,false_negative_rate_ratio,0.882588
8,false_omission_rate_ratio,1.053395
9,false_discovery_rate_ratio,1.046415


# Adversarial Fairness

## Method

## Pros and Cons

## Materials

In [6]:
from sklearn.preprocessing import MinMaxScaler

def collapse_group(x):
    if x!=3:
        return 0
    else:
        return 1

train['race'] = train['race'].apply(collapse_group)
test['race'] = test['race'].apply(collapse_group)

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 0}],
                   'privileged_groups':[{'race': 1}]
                  }

scaler = MinMaxScaler()
train[FEATURE_COLS] = scaler.fit_transform(train[FEATURE_COLS])
test[FEATURE_COLS] = scaler.transform(test[FEATURE_COLS])

In [7]:
from aif360.algorithms.inprocessing import AdversarialDebiasing
import tensorflow as tf

ground_truth_train = make_dataset(train[FEATURE_COLS], train['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)
ground_truth_test = make_dataset(test[FEATURE_COLS], test['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)

sess = tf.Session()

model_params = {'classifier_num_hidden_units':200, 
                'num_epochs':50,
                'batch_size':128,
                'adversary_loss_weight':0.1,
                'debias':True
               }

clf = AdversarialDebiasing(scope_name='debiased_classifier', sess=sess, seed=1234, 
                           **PRIVILEGED_INFO,
                           **model_params
                           )

clf.fit(ground_truth_train)
prediction_test = clf.predict(ground_truth_test)

roc_auc = roc_auc_score(test['two_year_recid'], prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





epoch 0; iter: 0; batch classifier loss: 0.685280; batch adversarial loss: 0.803090
epoch 1; iter: 0; batch classifier loss: 0.678185; batch adversarial loss: 0.750131
epoch 2; iter: 0; batch classifier loss: 0.672976; batch adversarial loss: 0.750074
epoch 3; iter: 0; batch classifier loss: 0.684506; batch adversarial loss: 0.715449
epoch 4; iter: 0; batch classifier loss: 0.607757; batch adversarial loss: 0.762628

In [8]:
joblib.dump((clf_metric,roc_auc), '../results/1.3-adversarial_fairness.pkl')
display_results('../results/1.3-adversarial_fairness.pkl')

,metric_names,scores
0,roc_auc_score,0.659522
1,true_positive_rate_difference,0.172664
2,false_positive_rate_difference,0.129211
3,false_omission_rate_difference,-0.123946
4,false_discovery_rate_difference,0.051620
5,error_rate_difference,-0.040911
6,false_positive_rate_ratio,1.415630
7,false_negative_rate_ratio,0.562062
8,false_omission_rate_ratio,0.726193
9,false_discovery_rate_ratio,1.197017


In [11]:
sess.close()
tf.reset_default_graph()